In [1]:
"""새로운 Jupyter notebook에서 리서치 AI 에이전트를 만들고 커스텀 도구를 부여합니다.
에이전트는 다음 작업을 수행할 수 있어야 합니다:
Wikipedia에서 검색
DuckDuckGo에서 검색
웹사이트의 텍스트를 스크랩하고 추출합니다.
리서치 결과를 .txt 파일에 저장하기
다음 쿼리로 에이전트를 실행합니다: "Research about the XZ backdoor" 라는 쿼리로 에이전트를 실행하면, 에이전트는 Wikipedia 또는 DuckDuckGo에서 검색을 시도하고, DuckDuckGo에서 웹사이트를 찾으면 해당 웹사이트에 들어가서 콘텐츠를 추출한 다음 .txt 파일에 조사 내용을 저장하는 것으로 완료해야 합니다.
제출방법*/"""

import os
from pydantic import BaseModel, Field
from typing import Type
from langchain_openai import ChatOpenAI
from langchain.document_loaders.web_base import WebBaseLoader
from langchain.schema import SystemMessage
from langchain_community.agent_toolkits.file_management import FileManagementToolkit
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities.wikipedia import WikipediaAPIWrapper
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper


llm = ChatOpenAI(temperature=0.1)


class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the XZ backdoor"
    )


class WikiSearchTool(BaseTool):
    name = "WikiSearchTool"
    description = """
    Use this tool to find the search Result in this website with query arugment.
    Example query: Research about the XZ backdoor
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to find the search Result in this website with query arugment.
    Example query: Research about the XZ backdoor
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg_wrapper = DuckDuckGoSearchAPIWrapper()
        return ddg_wrapper.run(query)


class LoadWebsiteTool(BaseTool):
    name = "LoadWebsiteTool"
    description = """
    Use this tool to load the website for the given url.
    """

    class LoadWebsiteToolArgsSchema(BaseModel):
        url: str = Field(
            description="The url you will load. Example url: https://en.wikipedia.org/wiki/Backdoor_(computing)",
        )

    args_schema: Type[LoadWebsiteToolArgsSchema] = LoadWebsiteToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader([url])
        docs = loader.load()

        return docs


# 에이전트 초기화
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        DuckDuckGoSearchTool(),
        WikiSearchTool(),
        LoadWebsiteTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are a helful search manager assistance.
            
            You should try to search information WikiPedia or DuckDuckGo,
            
            If there is a list of website urls in the search results list, please extract the content of each website to text and organize it, 
            This will be documented, so please keep the results organized.
            
        """
        )
    },
)


working_directory = os.getcwd()

tools = FileManagementToolkit(
    root_dir=working_directory,
    selected_tools=["write_file"],
).get_tools()

(write_tool,) = tools

query = "Research about the XZ backdoor"
result = agent.invoke(query)
write_tool.invoke({"file_path": f"{query}.txt", "text": result["output"]})

/Users/byul/Documents/projects/nomad-python-study/FullStack-gpt/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/byul/Documents/projects/nomad-python-study/FullStack-gpt/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`


The XZ Backdoor: Everything You Need to Know. Details are starting to emerge about a stunning supply chain attack that sent the open source software community reeling. On Friday, a lone Microsoft ... The version of XZ should be carefully checked, as versions 5.6.0 and 5.6.1 contain the backdoor. It is advised to revert to a previous known safe version of XZ Utils, such as 5.4. Software supply ... The first piece of the backdoor is the m4/build-to-host.m4 file. This file orchestrates minor modifications and conceals the extraction and execution of the Stage 1 backdoor file, bad-3-corrupt_lzma2.xz. Note how the grep command matches one file in the source directory: Only one file matches given bytes for both versions A backdoor in upstream xz/liblzma was announced on the oss-security mailing list regarding the xz compression tools and libraries. Specifically, the issue with the xz l

'File written successfully to Research about the XZ backdoor.txt.'